In [13]:
import sys, os

sys.path.append(os.pardir)
from dataset.mnist import load_mnist
import numpy as np
from PIL import Image
import pickle
from common.functions import sigmoid, softmax

In [14]:

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()


# normalize:入力画像を0~1に正規化する。
# flatten:画像を1次元配列に変換する
# noe_hot_label:ラベルをone hot表現（0か1)にする
#(訓練画像、訓練ラベル),(テスト画像,テストラベル)を出力する
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)
img = x_train[0]
label = t_train[0]
print(label)

# 1次元の配列の次元数が表示
print(img.shape)
# 2次元に再変換
img = img.reshape(28, 28)
# 2次元配列の次元数が表示
print(img.shape)
img_show(img)


5
(784,)
(28, 28)


In [15]:



def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(
        normalize=True, flatten=True, one_hot_label=False
    )
    # テスト画像とラベルだけを返す
    return x_test, t_test


def init_network():
    with open("./ch03/sample_weight.pkl", "rb") as f:
        network = pickle.load(f)
    return network


def predict(network, x):
    W1, W2, W3 = network["W1"], network["W2"], network["W3"]
    b1, b2, b3 = network["b1"], network["b2"], network["b3"]
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)

    return y

# テスト画像とラベルを取得
x, t = get_data()
# 学習済みのパラメータを取得
network = init_network()
batch_size = 100
accuracy_cnt = 0
# 分類データが正確に分類できるかを推論する
for i in range(0,len(x),batch_size):
    # iからi+batch_sizeのデータを取得
    x_batch = x[i:i+batch_size]
    # テスト画像を学習モデルに適用し、分類させる。
    y_batch = predict(network, x_batch)
    # yリストの中から、最大の確率を取得
    # axis=1とすることで、多次元配列の1次元目の中から最大の物を抽出
    p = np.argmax(y_batch,axis=1)
    # 最大確率となるインデックスがラベルデータと一致するか判定
    #　p==t[i:i+batch_size]とすると、pに一致する部分がTrue,不一致がFalseとなる。
    # このようなboolの配列が返却され、そのsumを取るとTrueの個数として取得できる。
    accuracy_cnt+=np.sum(p==t[i:i+batch_size])

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))


Accuracy:0.9352
